In [78]:
import pandas as pd 
docs = pd.read_csv('bigDoctorFile.csv', dtype = object)
docs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2955262 entries, 0 to 2955261
Data columns (total 41 columns):
NPI                                                                  object
PAC ID                                                               object
Professional Enrollment ID                                           object
Last Name                                                            object
First Name                                                           object
Middle Name                                                          object
Suffix                                                               object
Gender                                                               object
Credential                                                           object
Medical school name                                                  object
Graduation year                                                      object
Primary specialty                                  

In [108]:
docInfo = docs.loc[ : , ['NPI', 'PAC ID','Last Name', 'First Name', 'Gender', 'Credential', 'Graduation year']]
 
docInfo['Graduation year'] = pd.to_numeric(docInfo['Graduation year'], errors = 'coerce')

In [110]:
pivot = docInfo.pivot_table(values = 'Graduation year', index = 'Gender',
                            columns= 'Credential', aggfunc = np.mean, fill_value = 0)
list(pivot.columns)

['AA',
 'AU',
 'CNA',
 'CNM',
 'CNS',
 'CP',
 'CSW',
 'DC',
 'DDM',
 'DDS',
 'DO',
 'DPM',
 'MD',
 'MNT',
 'NP',
 'OD',
 'OT',
 'PA',
 'PSY',
 'PT',
 'SCW']

In [112]:
#sort so when I drop duplicates I retain the data I need
#get rid of some unnecessary data

 
docInfo.sort_values('Graduation year', ascending = False)
docInfo.sort_values('Gender', ascending = True)
replacements = { 'MD': 1, 'DO': 1, 'NP': 1, 'CSW': 1, 'OD': 1, 'DC': 1, 'CNA': 1, 'PA': 1, 'AA' : 1,
                'AU': 1, 'CNM': 1,'CNS':1, 'CP':1, 'DDM': 1, 'DDS':1, 'DPM': 1, 'MNT': 1, 'OT': 1,
                'PSY': 1, 'PT': 1, 'SCW':1 }
docInfo['newCredential'] = docInfo['Credential'].replace(replacements)
docInfo.head()
Credmx = docInfo.groupby(['NPI', 'PAC ID']).newCredential.transform(max)
docInfo = docInfo.loc[docInfo.newCredential == Credmx]

In [113]:
docInfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1013606 entries, 1 to 2955257
Data columns (total 8 columns):
NPI                1013606 non-null object
PAC ID             1013606 non-null object
Last Name          1013594 non-null object
First Name         1013606 non-null object
Gender             1013606 non-null object
Credential         1013606 non-null object
Graduation year    1010554 non-null float64
newCredential      1013606 non-null float64
dtypes: float64(2), object(6)
memory usage: 69.6+ MB


In [66]:
import numpy as np
##check for duplicated NPI values and save their indices to a list
docs['dupeNPI']= docs['NPI'].duplicated()
x = docs[docs['dupeNPI'] == True].index.tolist()

##check for duplicated PAC values and save their indices to a list
docs['dupePAC']= docs['PAC ID'].duplicated()
y = docs[docs['dupePAC'] == True].index.tolist()

##check if both NPI and PAC are duplicated (useless info)
z = x + y
from collections import Counter, defaultdict
dupindex = Counter(z)

##return index values for rows where NPI and PAC are both duplicated
def invert_dict(d):
    d_inv = defaultdict(list)
    for k, v in dupindex.items():
        d_inv[v].append(k)
    return d_inv
d = dict(invert_dict(dupindex))
results = pd.Series(d)
indexes_to_drop = results.loc[2]

#remove rows with duplicate NIP and PAC
#not using .drop because of time/memory

indexes_to_keep = set(range(docs.shape[0])) - set(indexes_to_drop)
docs_sliced = docs.take(list(indexes_to_keep))
docs_sliced.head()

,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Middle Name,Suffix,Gender,Credential,Medical school name,...,Hospital affiliation CCN 5,Hospital affiliation LBN 5,Professional accepts Medicare Assignment,Reported Quality Measures,Used electronic health records,Committed to heart health through the Million Hearts® initiative.,2first,2last,dupeNPI,dupePAC
0,1487927612,4880850486,I20120726000331,HALL,ESTHER,S,NaN,F,NaN,LIFE CHIROPRACTIC COLLEGE - WEST,...,NaN,NaN,Y,NaN,NaN,NaN,False,False,False,False
1,1235146762,2365435336,I20040406000367,WHITE,BARBARA,L,NaN,F,CSW,OTHER,...,NaN,NaN,Y,NaN,NaN,NaN,False,False,False,False
2,1346282258,5395768527,I20060113000139,DAVIDSON,JOHN,A,NaN,M,CSW,OTHER,...,NaN,NaN,Y,NaN,NaN,NaN,False,False,False,False
3,1932283124,5193762862,I20050415000143,CAGEN,STEVEN,F,NaN,M,DC,SHERMAN COLLEGE OF STRAIGHT CHIROPRACTIC,...,NaN,NaN,Y,NaN,NaN,NaN,False,False,False,False
4,1902950462,7416123666,I20120110000522,ESPY,LEISHA,H,NaN,F,NaN,LIFE CHIROPRACTIC COLLEGE,...,NaN,NaN,M,NaN,NaN,NaN,False,False,False,False


In [10]:
docs_sliced.set_index(['NPI', 'PAC ID'], inplace=True)


,,Professional Enrollment ID,Last Name,First Name,Middle Name,Suffix,Gender,Credential,Medical school name,Graduation year,Primary specialty,...,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5,Professional accepts Medicare Assignment,Reported Quality Measures,Used electronic health records,Committed to heart health through the Million Hearts® initiative.
NPI,PAC ID,,,,,,,,,,,,,,,,,,,,,
1487927612,4880850486,I20120726000331,HALL,ESTHER,S,NaN,F,NaN,LIFE CHIROPRACTIC COLLEGE - WEST,2010,CHIROPRACTIC,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
1235146762,2365435336,I20040406000367,WHITE,BARBARA,L,NaN,F,CSW,OTHER,1992,CLINICAL SOCIAL WORKER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
1346282258,5395768527,I20060113000139,DAVIDSON,JOHN,A,NaN,M,CSW,OTHER,1999,CLINICAL SOCIAL WORKER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
1932283124,5193762862,I20050415000143,CAGEN,STEVEN,F,NaN,M,DC,SHERMAN COLLEGE OF STRAIGHT CHIROPRACTIC,1997,CHIROPRACTIC,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
1902950462,7416123666,I20120110000522,ESPY,LEISHA,H,NaN,F,NaN,LIFE CHIROPRACTIC COLLEGE,1985,CHIROPRACTIC,...,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2955262 entries, (1487927612, 4880850486) to (1689931438, 2466759253)
Data columns (total 5 columns):
Last Name          object
First Name         object
Gender             object
Credential         object
Graduation year    object
dtypes: object(5)
memory usage: 151.6+ MB


TypeError: cannot do positional indexing on <class 'pandas.core.indexes.multi.MultiIndex'> with these indexers [NPI] of <class 'str'>

In [ ]:
keepCols = ['NPI', 'PAC ID', 'Last Name', 'First Name', 'Gender', 'Credential', 'Graduation year']
docInfo = docs.loc[ : , ['keepCols']]
docInfo.head()